# 단어의 표현 (Word Representation)


기계는 문자를 그대로 인식할 수 없기때문에 숫자로 변환



#1 TF-IDF를 활용한 단어 벡터

##1-1 직접 구현하기

weighting schema|weight|설명
--|--|--
term frequency|<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/91699003abf4fe8bdf861bbce08e73e71acf5fd4" />|=토큰빈도/문서내토큰빈도
inverse document frequency|<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/864fcfdc0c16344c11509f724f1aa7081cf9f657" />|=log(총문서갯수/(토큰이 등장한 문서수))

In [1]:
d1 = "The cat sat on my face I hate a cat"
d2 = "The dog sat on my bed I love a dog" 

In [2]:
import numpy as np

# document 내 토큰이 등장한 빈도수 계산
def f(t, d):
    return d.count(t)

# tf 계산
def tf(t, d):
    return f(t,d)/len(d)

# idf 계산
def idf(t, D):
    numerator = len(D)
    denominator = len([True for d in D if t in d])
    return np.log(numerator/denominator)

# tf-idf 계산
def tfidf(t, d, D):
    return tf(t,d)*idf(t, D)

# 공백을 기준으로 토큰과
def tokenizer(d):
    return d.split()

# tfidf 계산  
def tfidfScorer(D):
    docs = [tokenizer(d) for d in D]
    result = []
    for d in docs:
        result.append([(t, tfidf(t, d, docs)) for t in d])
    return result

corpus = [d1, d2]

for i, doc in enumerate(tfidfScorer(corpus)):
    print('====== document[%d] ======' % i)
    print(doc)

====== document[0] ======
[('The', 0.0), ('cat', 0.13862943611198905), ('sat', 0.0), ('on', 0.0), ('my', 0.0), ('face', 0.06931471805599453), ('I', 0.0), ('hate', 0.06931471805599453), ('a', 0.0), ('cat', 0.13862943611198905)]
====== document[1] ======
[('The', 0.0), ('dog', 0.13862943611198905), ('sat', 0.0), ('on', 0.0), ('my', 0.0), ('bed', 0.06931471805599453), ('I', 0.0), ('love', 0.06931471805599453), ('a', 0.0), ('dog', 0.13862943611198905)]


## 1-2 sklearn 활용

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

 
d1 = "The cat sat on my face I hate a cat"
d2 = "The dog sat on my bed I love a dog"
corpus = [d1, d2]
count_vect = CountVectorizer()
countv = count_vect.fit_transform(corpus)
print(countv.toarray()) # 코퍼스로부터 각 단어의 빈도 수를 기록한다.
print(count_vect.vocabulary_) # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

[[0 2 0 1 1 0 1 1 1 1]
 [1 0 2 0 0 1 1 1 1 1]]
{'the': 9, 'cat': 1, 'sat': 8, 'on': 7, 'my': 6, 'face': 3, 'hate': 4, 'dog': 2, 'bed': 0, 'love': 5}


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

d1 = "The cat sat on my face I hate a cat"
d2 = "The dog sat on my bed I love a dog"
corpus = [d1, d2]
tfidf_vect = TfidfVectorizer().fit(corpus)
tfidfv = tfidf_vect.transform(corpus)
print(tfidfv.toarray())
print(tfidf_vect.vocabulary_)

[[0.         0.70600557 0.         0.35300279 0.35300279 0.
  0.25116439 0.25116439 0.25116439 0.25116439]
 [0.35300279 0.         0.70600557 0.         0.         0.35300279
  0.25116439 0.25116439 0.25116439 0.25116439]]
{'the': 9, 'cat': 1, 'sat': 8, 'on': 7, 'my': 6, 'face': 3, 'hate': 4, 'dog': 2, 'bed': 0, 'love': 5}


## 1-3 gensim 활용

In [6]:
from gensim.models import TfidfModel
from gensim import corpora

d1 = "The cat sat on my face I hate a cat"
d2 = "The dog sat on my bed I love a dog"
corpus = [d1, d2]

doc_ls = [doc.split() for doc in corpus]
id2word = corpora.Dictionary(doc_ls)  # fit dictionary
corpus = [id2word.doc2bow(doc) for doc in doc_ls]  # convert corpus to BoW format

tfidf = TfidfModel(corpus)  # fit model
vector = tfidf[corpus[0]]  # apply model to the first corpus document

In [8]:
corpus

[[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)],
 [(0, 1), (1, 1), (2, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1)]]

In [9]:
tfidf[corpus[0]]

[(3, 0.8164965809277261), (4, 0.4082482904638631), (5, 0.4082482904638631)]



---

